In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

os.chdir("../")

import shfl
import torch
import copy
import numpy as np
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer

from shfl.private import UnprotectedAccess
from CIT.model import CITModel
from utils import get_federated_data_csv, get_transformed_data
from ClassifierModel import ClassifierModel

/mnt/sdd/fcastro/envs/TF/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/mnt/sdd/fcastro/envs/TF/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/mnt/sdd/fcastro/envs/TF/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/mnt/sdd/fcastro/envs/TF/lib/python3.6/site-packages/tensorflow/python/frame

In [2]:
#"../data/COVIDGR1.0/centralized/cropped"
#partition_iid_3nodes_1.csv
args = {"data_path":"../data/Revisadas-Clasificadas-Recortadas", 
        "csv_path": "../partitions/partition_iid_1nodes_2.csv",
        "output_path": "../weights",
        "input_path": "",
        "model_name":"transferlearning.model", 
        "label_bin": "lb.pickle", 
        "batch_size": 8,
        "federated_rounds": 1,
        "epochs_per_FL_round": 1,
        "num_nodes": 3,
        "size_averaging": 1,
        "random_rotation": 0,
        "random_shift": 0, 
        "random_zoom": 0,
        "horizontal_flip": False,        
        "finetune": True,
        "train_network": True}

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = 'cpu'

In [3]:
a = ['N', 'P']
b = ['NTN', 'NTP', 'PTP', 'PTN']
lb1 = LabelBinarizer()
lb2 = LabelBinarizer()
lb1.fit(a)
lb2.fit(b)

dict_labels = { 'PTP' : np.argmax(lb2.transform(['PTP'])[0]) , 'PTN' : np.argmax(lb2.transform(['PTN'])[0]) , 
                'NTP' : np.argmax(lb2.transform(['NTP'])[0]) , 'NTN' : np.argmax(lb2.transform(['NTN'])[0]), 
                'P' : lb1.transform(['P'])[0][0], 'N' : lb1.transform(['N'])[0][0]
              } 



In [4]:
def cit_builder():    
    return CITModel(['N', 'P'], classifier_name = "resnet18", lambda_values = [0.05], batch_size=args["batch_size"], epochs=args["epochs_per_FL_round"], device=device)

def classifier_builder( G_dict ):
    return ClassifierModel(G_dict, dict_labels, batch_size=args["batch_size"], epochs=args["epochs_per_FL_round"], finetune = args["finetune"])

In [5]:
print("[INFO] Fetching federated data...")
federated_data, train_data, train_label, test_data, test_label, train_files, test_files, args["num_nodes"] = get_federated_data_csv(args["data_path"], args["csv_path"], lb1)
federated_data.configure_data_access(UnprotectedAccess())
print(len(train_data))
print(len(test_data))
print("[INFO] done")

[INFO] Fetching federated data...
681
171
[INFO] done


In [6]:
aggregator = shfl.federated_aggregator.FedAvgAggregator()
cit_federated_government = shfl.federated_government.FederatedGovernment(cit_builder, federated_data, aggregator)
cit_federated_government.run_rounds(args["federated_rounds"], test_data, test_label)

Accuracy round 0
Training node 0
[INFO] weights = [1.         0.92917847]


[Validating]: Acc_D: 0.4348: 100%|██████████| 69/69 [00:02<00:00, 30.31it/s]


[INFO] Initial Valid Scores: 
Valid Acc = 0.43478260869565216
Valid Loss = 0.7608250958332117
[INFO] LAMBDA: 0.05


[1/1] Loss_D: 0.3607 Acc_D: 0.6070 Loss_G_class1: 0.0657 Loss_G_class2: 0.0582: 100%|██████████| 77/77 [01:51<00:00,  1.44s/it]
  0%|          | 0/69 [00:00<?, ?it/s]


Valid Acc = 0.7681159420289855
Valid Loss = 0.5452676468893237


[Validating]: Acc_D: 0.7536: 100%|██████████| 69/69 [00:01<00:00, 49.52it/s]


[INFO] Summary of training for LAMBDA = 0.05 (best model values)
Valid Acc = 0.7536231884057971
Valid Loss = 0.5587751281218253
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x7f1aeb406470>: [0.5026990312385803, 0.8187134502923976, {'0': {'precision': 0.9795918367346939, 'recall': 0.768, 'f1-score': 0.8609865470852017, 'support': 125}, '1': {'precision': 0.6027397260273972, 'recall': 0.9565217391304348, 'f1-score': 0.7394957983193277, 'support': 46}, 'accuracy': 0.8187134502923976, 'macro avg': {'precision': 0.7911657813810455, 'recall': 0.8622608695652174, 'f1-score': 0.8002411727022647, 'support': 171}, 'weighted avg': {'precision': 0.8782164151409181, 'recall': 0.8187134502923976, 'f1-score': 0.8283048251949667, 'support': 171}}]
Global model test performance : [0.5026990312385803, 0.8187134502923976, {'0': {'precision': 0.9795918367346939, 'recall': 0.768, 'f1-score': 0.8609865470852017, 'support': 125}, '1': {'precision': 0.6027397260273972,

[[0.5026990312385803,
  0.8187134502923976,
  {'0': {'precision': 0.9795918367346939,
    'recall': 0.768,
    'f1-score': 0.8609865470852017,
    'support': 125},
   '1': {'precision': 0.6027397260273972,
    'recall': 0.9565217391304348,
    'f1-score': 0.7394957983193277,
    'support': 46},
   'accuracy': 0.8187134502923976,
   'macro avg': {'precision': 0.7911657813810455,
    'recall': 0.8622608695652174,
    'f1-score': 0.8002411727022647,
    'support': 171},
   'weighted avg': {'precision': 0.8782164151409181,
    'recall': 0.8187134502923976,
    'f1-score': 0.8283048251949667,
    'support': 171}}]]

In [7]:
metrics = cit_federated_government.global_model.evaluate(test_data, test_label)
print("CIT Classifier Results:")
print("Loss: {}".format(metrics[0]))
print("Acc: {}".format(metrics[1]))
print(metrics[2])

CIT Classifier Results:
Loss: 0.5026990312385803
Acc: 0.8187134502923976
{'0': {'precision': 0.9795918367346939, 'recall': 0.768, 'f1-score': 0.8609865470852017, 'support': 125}, '1': {'precision': 0.6027397260273972, 'recall': 0.9565217391304348, 'f1-score': 0.7394957983193277, 'support': 46}, 'accuracy': 0.8187134502923976, 'macro avg': {'precision': 0.7911657813810455, 'recall': 0.8622608695652174, 'f1-score': 0.8002411727022647, 'support': 171}, 'weighted avg': {'precision': 0.8782164151409181, 'recall': 0.8187134502923976, 'f1-score': 0.8283048251949667, 'support': 171}}


In [8]:
t_federated_data, t_test_data, t_test_label = get_transformed_data(federated_data, cit_federated_government, test_data, test_label, lb1, lb2)

In [9]:
aggregator = shfl.federated_aggregator.FedAvgAggregator()
G_dict = cit_federated_government.global_model._G_dict
x = lambda : classifier_builder(G_dict)
classifier_federated_government = shfl.federated_government.FederatedGovernment(x, t_federated_data, aggregator)
#classifier_federated_government = FederatedGovernment_2(classifier_builder(G_dict), t_federated_data, aggregator)
classifier_federated_government.run_rounds(args["federated_rounds"], t_test_data, t_test_label)

#print("[INFO] saving model ...")
#federated_government.global_model._model.save( os.path.join(args["output_path"], args["model_name"]) )
print("[INFO] done")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Accuracy round 0
Training node 0
153/153 [==============================] - 28s 186ms/step - loss: 0.7510 - categorical_accuracy: 0.6429 - val_loss: 0.7957 - val_categorical_accuracy: 0.6324
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x7f1ae26e7438>: [0.5789473684210527, 0.5350877192982456, 58, {'0': {'precision': 0.825, 'recall': 0.336734693877551, 'f1-score': 0.47826086956521735, 'support': 98}, '1': {'precision': 0.5038167938931297, 'recall': 0.9041095890410958, 'f1-score': 0.6

In [10]:
metrics = classifier_federated_government.global_model.evaluate2(test_data, test_label)
print("SDNET Classifier Results:")
print("Acc: {}".format(metrics[0]))
print("Acc_4: {}".format(metrics[1]))
print("No concuerda: {}".format(metrics[2]))
print(metrics[3])

SDNET Classifier Results:
Acc: 0.5789473684210527
Acc_4: 0.5350877192982456
No concuerda: 58
{'0': {'precision': 0.825, 'recall': 0.336734693877551, 'f1-score': 0.47826086956521735, 'support': 98}, '1': {'precision': 0.5038167938931297, 'recall': 0.9041095890410958, 'f1-score': 0.6470588235294118, 'support': 73}, 'accuracy': 0.5789473684210527, 'macro avg': {'precision': 0.6644083969465648, 'recall': 0.6204221414593234, 'f1-score': 0.5626598465473146, 'support': 171}, 'weighted avg': {'precision': 0.6878867014865407, 'recall': 0.5789473684210527, 'f1-score': 0.5503208148247857, 'support': 171}}


In [11]:
metrics = classifier_federated_government.global_model.evaluate(t_test_data, t_test_label)
print("SDNET Classifier Results:")
print("Acc: {}".format(metrics[0]))
print("Acc_4: {}".format(metrics[1]))
print("No concuerda: {}".format(metrics[2]))
print(metrics[3])

SDNET Classifier Results:
Acc: 0.5789473684210527
Acc_4: 0.5350877192982456
No concuerda: 58
{'0': {'precision': 0.825, 'recall': 0.336734693877551, 'f1-score': 0.47826086956521735, 'support': 98}, '1': {'precision': 0.5038167938931297, 'recall': 0.9041095890410958, 'f1-score': 0.6470588235294118, 'support': 73}, 'accuracy': 0.5789473684210527, 'macro avg': {'precision': 0.6644083969465648, 'recall': 0.6204221414593234, 'f1-score': 0.5626598465473146, 'support': 171}, 'weighted avg': {'precision': 0.6878867014865407, 'recall': 0.5789473684210527, 'f1-score': 0.5503208148247857, 'support': 171}}


In [12]:
print(test_label)

[[0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]]
